# Sonde3 
##  Reads and converts binary water quality environmental instrument data to a DataFrame


### I.  Example Usage

Lets dive in!  

We have a example water quality instrument binary file `"tests/ysi_test_files/SA08.dat"`.  This file was generated by a YSI 600LS instrument and is in proprietary binary format.

#### Using the `sonde()` function we:

1.  `autodetect()` the file type and pass to the correct parser function 
2.  `read_ysi()` the binary file and convert to pandas DataFrame
3.  Transform all datetimes to the UTC timezone
4.  Standardize the units to metric and rename the columns to standard name conventions
3.  Pass the DataFrame to `calculate_salinity_psu()` and `calculate_do_mgl()` to apply standard formulas to generate the salinity and dissolved oxygen columns.

In [1]:
import sonde3
import pandas
%matplotlib inline
metadata, df = sonde3.sonde("tests/ysi_test_files/SA08.dat", remove_invalids=False, twdbparams=True)

C:\Users\ETurner\Desktop\python\sonde3\sonde3\sonde.py:20: UserWarning: Info: No time zone was set for file, assuming records are recorded in CST
  metadata, df = formats.read_ysi(filename, tzinfo)
C:\Users\ETurner\AppData\Local\Continuum\Anaconda3\lib\site-packages\seawater\library.py:363: RuntimeWarning: invalid value encountered in sqrt
  Rtx = (rt) ** 0.5


#### Why the runtime warnings?

1.  The YSI instrument files don't contain any timezone information.  Therefore, the function has to assume that the timezone of the file to make the UTC conversion.

2. Often raw instrument files will contain impossible & incorrect values in the beginning and end of the file.  Examples: negative values for salinity or dissolved oxygen percentage.  `sonde3` does not trim the raw file, or perform QA analysis.  `sonde3` will pass the values as they were recorded by the instrument.

To automatically convert invalid negative values to zero, either set the flag **remove_invalids=False** or remove it altogether, as this is the default behavior of the package

##### We can now interact with the two dataframes produced by `sonde3`:


In [2]:
df.info() 
df.head() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 9 columns):
Datetime_(UTC)                               700 non-null datetime64[ns, UTC]
water_temperature                            700 non-null float64
water_electrical_conductivity                700 non-null float64
water_depth_nonvented                        700 non-null float64
water_dissolved_oxygen_percent_saturation    700 non-null float64
instrument_battery_voltage                   700 non-null float64
water_specific_conductance                   700 non-null float64
seawater_salinity                            678 non-null float64
water_dissolved_oxygen_concentration         678 non-null float64
dtypes: datetime64[ns, UTC](1), float64(8)
memory usage: 49.3 KB


,Datetime_(UTC),water_temperature,water_electrical_conductivity,water_depth_nonvented,water_dissolved_oxygen_percent_saturation,instrument_battery_voltage,water_specific_conductance,seawater_salinity,water_dissolved_oxygen_concentration
0,2008-07-16 12:00:31+00:00,28.998718,0.000037,0.010862,93.391418,6.09375,0.000035,0.013536,7.183420
1,2008-07-16 13:00:31+00:00,28.482361,0.000059,0.016358,96.765137,6.09375,0.000055,0.013326,7.510631
2,2008-07-16 14:00:31+00:00,27.257385,0.000546,0.017263,103.529358,6.09375,0.000524,0.012655,8.212117
3,2008-07-16 15:00:31+00:00,29.507751,21.301758,0.542648,93.055725,6.09375,19.613107,11.601472,6.655432
4,2008-07-16 16:00:31+00:00,29.762268,21.454102,0.557098,94.188690,6.09375,19.665354,11.631321,6.706414


In [3]:
metadata

,Model,Manufacturer,Instrument_Serial_Number,Station,Deployment_Setup_Time,Filename,Deployment_Start_Time,Deployment_Stop_Time
0,600,YSI,1012,SANT_CDT,,SA08.dat,2008-07-16 12:00:31+00:00,2008-08-14 15:00:16+00:00


### II.  Working with time zones


What if data was collected outside of US/Central time?  Pass the timezone information to `sonde3.sonde`:

In [4]:
import pytz
metadata, df = sonde3.sonde("tests/ysi_test_files/SA08.dat", pytz.timezone('US/Eastern'))
df.head()

,Datetime_(UTC),water_temp_C,water_conductivity_mS/cm,water_depth_m_nonvented,water_DO_%,instrument_battery_voltage,water_specific_conductivity_mS/cm,water_salinity_PSU,water_DO_mgl
0,2008-07-16 11:00:31+00:00,28.998718,0.000037,0.010862,93.391418,6.09375,0.000035,0.013536,7.183420
1,2008-07-16 12:00:31+00:00,28.482361,0.000059,0.016358,96.765137,6.09375,0.000055,0.013326,7.510631
2,2008-07-16 13:00:31+00:00,27.257385,0.000546,0.017263,103.529358,6.09375,0.000524,0.012655,8.212117
3,2008-07-16 14:00:31+00:00,29.507751,21.301758,0.542648,93.055725,6.09375,19.613107,11.601472,6.655432
4,2008-07-16 15:00:31+00:00,29.762268,21.454102,0.557098,94.188690,6.09375,19.665354,11.631321,6.706414


### III. Autodetecting files



Curious about what kind of instrument files you have in a directory?  Apply the `sonde3.autodetect` method:

```python
#what kind of file is this??
sonde3.autodetect("tests/greenspan_test_files/RIOA_20060718_CDT_GS7837.xls") 
```

In [5]:
sonde3.autodetect("tests/greenspan_test_files/RIOA_20060718_CDT_GS7837.xls") 

'greenspan_xls'

Copy this code snippet to the notebook line below to run `autodetect()` on all of the use-case examples in the `sonde3` package:

```python
#this script runs through all of the text examples and prints out the autodetect results
import os

root_dir = 'tests'
results = []
for directory, subdirectories, files in os.walk(root_dir):
    for file in files:
        if "_test.txt" in file:
            continue
        os.path.join(directory, file)
        results.append(os.path.join(directory, file) + ' ' + sonde3.autodetect(os.path.join(directory, file)))

results 
```

### IV. Generating Salinity and Dissolved Oxygen

Typically deployed water quality instruments do not compute all rows of data internally.  Instead, these are calculated by the program used to read the file back at the lab.  For example, YSI instruments do not compute salinity or dissolved oxygen concentration.  This can cause confusion because when viewing a YSI *\*.dat* file in YSI's ECOWIN, or Ecowatch Lite program displays salinity and DO mg/L.  However, the raw binary file does not include these rows as they were not physically collected by the instrument during deployment.

For example, lets read the raw binary file of the example file `"tests/ysi_test_files/SA08.dat"` and see what it contains:

In [6]:
metadata, SA08_BIN = sonde3.read_ysi("tests/ysi_test_files/SA08.dat",pytz.timezone('US/Central'))
SA08_BIN.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 6 columns):
Datetime_(UTC)                700 non-null datetime64[ns, UTC]
water_temp_C                  700 non-null float64
water_conductivity_mS/cm      700 non-null float64
water_depth_m_nonvented       700 non-null float64
water_DO_%                    700 non-null float64
instrument_battery_voltage    700 non-null float64
dtypes: datetime64[ns, UTC](1), float64(5)
memory usage: 32.9 KB


The pandas DataFrame contains the columns for conductivity and % DO saturation, but not for salinity and DO concentration.

For comparision, lets read the comma separated version of this file that was produced by the proprietary YSI Ecowin program:

In [7]:
metadata, SA08_CSV = sonde3.read_ysi_ascii("tests/ysi_test_files/SA08.CDF", pytz.timezone('US/Central'),delim=",")


The ECOWIN exported comma separated file has far more columns! The extra columns were derived through formulas using the instrumnet observed measurments.

To generate these columns pass the SA08_BIN DataFrame to calculate Salinity (PSU) and Dissolved Oxygen (mg/L) using `sonde3.calculate_salinity_psu()` and `sonde3.calculate_do_mgl()`

We can then compare our computed results to the ECOwatch program results:

In [8]:
SA08_BIN = sonde3.calculate_salinity_psu(SA08_BIN)
SA08_BIN = sonde3.calculate_do_mgl(SA08_BIN)
SA08_BIN.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 8 columns):
Datetime_(UTC)                700 non-null datetime64[ns, UTC]
water_temp_C                  700 non-null float64
water_conductivity_mS/cm      700 non-null float64
water_depth_m_nonvented       700 non-null float64
water_DO_%                    700 non-null float64
instrument_battery_voltage    700 non-null float64
water_salinity_PSU            678 non-null float64
water_DO_mgl                  678 non-null float64
dtypes: datetime64[ns, UTC](1), float64(7)
memory usage: 43.8 KB


C:\Users\ETurner\AppData\Local\Continuum\Anaconda3\lib\site-packages\seawater\library.py:363: RuntimeWarning: invalid value encountered in sqrt
  Rtx = (rt) ** 0.5


So, why does `sonde3` produce less values for salinity and DO than the comma separated file?  Is this a bug?

*NO!*  If you recal from the `sqrt()` warning above, some of the values in the file are invalid.  This produces null (**NaN**) values in those rows.

In [9]:
SA08_BIN.tail()

,Datetime_(UTC),water_temp_C,water_conductivity_mS/cm,water_depth_m_nonvented,water_DO_%,instrument_battery_voltage,water_salinity_PSU,water_DO_mgl
695,2008-08-14 11:00:16+00:00,23.977051,-0.000006,-0.052356,-0.006866,5.546875,NaN,NaN
696,2008-08-14 12:00:16+00:00,23.938599,-0.000005,-0.050262,-0.006866,5.468750,NaN,NaN
697,2008-08-14 13:00:16+00:00,24.039307,-0.000005,-0.045875,-0.006866,5.468750,NaN,NaN
698,2008-08-14 14:00:16+00:00,24.107056,-0.000005,-0.038671,-0.006866,5.468750,NaN,NaN
699,2008-08-14 15:00:16+00:00,24.177551,-0.000006,-0.034785,-0.006866,5.468750,NaN,NaN


Lets check our two files to see if our conversion methods are those used by YSI.  

Lets use `numpy.random` to pick a row to check:



In [10]:
from numpy import random
row = random.randint(2,677) # pick random row

sub1 =  SA08_BIN.iloc[row:row+1]  #row one binary file
sub2 = SA08_CSV.iloc[row:row+1]   #row two csv file

pandas.concat([sub1, sub2], axis=0,join="inner")

,Datetime_(UTC),water_temp_C,water_conductivity_mS/cm,water_depth_m_nonvented,water_DO_%,instrument_battery_voltage,water_salinity_PSU,water_DO_mgl
419,2008-08-02 23:00:31+00:00,31.499939,37.731445,0.551926,100.675964,5.859375,20.867863,6.621754
419,2008-08-02 23:00:31+00:00,31.500000,37.731000,0.552000,100.700000,5.900000,20.870000,6.620000


Looks the same!  Notice, however, that the csv file values are rounded due to the fixed digit precision.

### V. Files from other Manufacturers and Models
#### a. Hydrotech

In [11]:
metadata, mydf = sonde3.sonde("tests/hydrotech_test_files/0109DELT.CSV", pytz.timezone('US/Central'))

C:\Users\ETurner\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [12]:
metadata

,Manufacturer,Instrument_Serial_Number,Model,Station,Deployment_Setup_Time,Deployment_Start_Time,Deployment_Stop_Time,Filename
0,Hydrotech,081107-D,MiniSonde4a,0109delt,2009-01-13 06:00:00+00:00,2009-01-14 13:00:00+00:00,2009-02-12 16:00:00+00:00,NaN


In [13]:
mydf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 700 entries, 13 to 712
Data columns (total 7 columns):
Datetime_(UTC)                       700 non-null datetime64[ns, UTC]
water_temp_C                         700 non-null float64
water_specific_conductivity_mS/cm    700 non-null float64
water_depth_m_nonvented              700 non-null float64
instrument_battery_voltage           700 non-null float64
water_conductivity_mS/cm             700 non-null float64
water_salinity_PSU                   700 non-null float64
dtypes: datetime64[ns, UTC](1), float64(6)
memory usage: 43.8 KB


In [14]:
mydf.head()

,Datetime_(UTC),water_temp_C,water_specific_conductivity_mS/cm,water_depth_m_nonvented,instrument_battery_voltage,water_conductivity_mS/cm,water_salinity_PSU
13,2009-01-14 13:00:00+00:00,12.81,0.866,-0.0,6.0,0.664370,0.428305
14,2009-01-14 14:00:00+00:00,12.57,0.868,-0.0,6.0,0.661926,0.429304
15,2009-01-14 15:00:00+00:00,12.34,0.867,-0.0,6.0,0.657354,0.428759
16,2009-01-14 16:00:00+00:00,12.59,0.868,-0.0,6.0,0.662257,0.429306
17,2009-01-14 17:00:00+00:00,11.78,0.883,-0.0,6.0,0.660041,0.436875


In [15]:
mydf.tail()

,Datetime_(UTC),water_temp_C,water_specific_conductivity_mS/cm,water_depth_m_nonvented,instrument_battery_voltage,water_conductivity_mS/cm,water_salinity_PSU
708,2009-02-12 12:00:00+00:00,19.24,34.9,1.03,3.9,31.060442,22.008874
709,2009-02-12 13:00:00+00:00,19.22,34.9,1.02,3.9,31.047110,22.008973
710,2009-02-12 14:00:00+00:00,19.31,35.1,1.02,3.9,31.285367,22.147645
711,2009-02-12 15:00:00+00:00,19.44,35.8,1.02,3.9,31.998183,22.634720
712,2009-02-12 16:00:00+00:00,19.34,34.9,1.03,0.0,31.127101,22.008374


#### b. YSI EXO Units

YSI EXO units are timezone aware.  Thus, we discover the internal timezone and convert to UTC if applicable.

In [16]:
sonde3.autodetect("tests/ysi_test_files/GE-SA-B_17H104157_090617_060000.csv")

'ysi_exo_csv'

In [17]:
metadata, exo_df = sonde3.sonde("tests/ysi_test_files/GE-SA-B_17H104157_090617_060000.csv")

C:\Users\ETurner\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [18]:
metadata

,Manufacturer,Instrument_Serial_Number,Sensor_Serial_Numbers,Model,Station,Deployment_Setup_Time,Deployment_Start_Time,Deployment_Stop_Time,Filename,User,Averaging,Firmware,Sensor_Firmware
0,YSI,17H104157,17H102123;17G101402;17H101513;17H100181,EXO,GEB,NaN,2017-09-06 11:15:01,2017-09-26 19:45:00,NaN,Evan Turner,Default,3.0.5,3.0.0;3.0.5;3.0.0;3.0.0


In [19]:
exo_df.head()

,Datetime_(UTC),sonde_fault_code,instrument_battery_voltage,sonde_cable_power_voltage,water_DO_%,water_DO_mgl,water_temp_C,water_nLF_conductivity_uS/cm,water_tds_mgl,water_pressure_abs,water_depth_m_nonvented,water_conductivity_mS/cm,water_specific_conductivity_mS/cm,water_salinity_PSU
21,2017-09-06 11:15:01,0.0,3.1,1.7,100.1,8.091890,26.205,0.0,0.0,0.044,0.031,0.0,0.0000,0.012744
22,2017-09-06 11:30:01,0.0,3.1,1.7,100.0,8.093779,26.137,0.0,0.0,0.043,0.030,0.0,0.0000,0.012720
23,2017-09-06 11:45:01,0.0,3.1,1.7,100.0,8.132112,25.877,0.0,0.0,0.046,0.032,0.0,0.0000,0.012626
24,2017-09-06 12:00:01,0.0,3.1,1.7,99.9,8.167665,25.583,-0.1,0.0,0.049,0.035,-0.0,-0.0001,0.012519
25,2017-09-06 12:15:01,0.0,3.1,1.7,99.9,8.198972,25.374,0.0,0.0,0.049,0.035,-0.0,0.0000,0.012443


#### c. YSI ascii + Comma Separated.

YSI timeseries produced by kermit transfer from YSI handset protocol.

In [20]:
metadata, df = sonde3.sonde("tests/ysi_test_files/0917GEB.txt")

C:\Users\ETurner\Desktop\python\sonde3\sonde3\sonde.py:26: UserWarning: Info: No time zone was set for file, assuming records are recorded in CST
  metadata, df = formats.read_ysi_ascii(filename,  tzinfo,',',None,[1,2,3])


In [21]:
metadata

,Manufacturer,Instrument_Serial_Number,Model,Station,Deployment_Setup_Time,Deployment_Start_Time,Deployment_Stop_Time
0,YSI,,,,,2017-09-08 12:15:08+00:00,2017-09-26 20:00:08+00:00


In [22]:
df.head()

,Datetime_(UTC),water_temp_C,water_tds_g/L,water_pressure_abs,water_depth_m_nonvented,instrument_battery_voltage,water_conductivity_mS/cm,water_specific_conductivity_mS/cm,water_salinity_PSU
0,2017-09-08 12:15:08+00:00,19.93,0.0,14.763,0.303,6.2,0.0,0.0,0.010285
1,2017-09-08 12:30:08+00:00,20.05,0.0,14.767,0.306,6.2,0.0,0.0,0.010336
2,2017-09-08 12:45:08+00:00,20.54,0.0,14.769,0.307,6.2,0.0,0.0,0.010544
3,2017-09-08 13:00:08+00:00,20.91,0.0,14.771,0.309,6.2,0.0,0.0,0.010699
4,2017-09-08 13:15:08+00:00,21.21,0.0,14.774,0.311,6.2,0.0,0.0,0.010823


#### D. Lowell Tiltmeter Comma Separated.

Concatinated timeseries file from a Lowell tiltmeter

In [23]:
metadata, df = sonde3.sonde("tests/lowell_test_files/1603103_SAB_05122016.csv")

C:\Users\ETurner\Desktop\python\sonde3\sonde3\sonde.py:34: UserWarning: Info: No time zone was set for file, assuming records are recorded in CST
  metadata, df = formats.read_lowell(filename,  tzinfo,',')
C:\Users\ETurner\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [24]:
sonde3.merge_lowell()

got one


In [25]:
metadata.head()

,Manufacturer,Instrument_Serial_Number,Model,Station,Deployment_Setup_Time,Deployment_Start_Time,Deployment_Stop_Time,Filename
0,Lowell,1603103,TCM,NaN,NaN,2016-05-13 17:30:00+00:00,2016-07-15 18:00:00+00:00,NaN


### VI. Package Validation Tests

#### A.  Check to see if the package is correctly handling daylight savings time.

The test case dataset **tests/ysi_test_files/0108BAYT.csv** was collected during a daylight savings transition.  These three files were processed differently at the time of instrument processing.  The *.dat* file is the raw binary file from the instrument.  The *CDF* file was produced by the Ecowatch program.  The *cvs* was produced by converting the *CDF* file in ms excel to a comma separated file.  If the times were not consistant between the three files it would indicate **sonde3** is not processing datetime correctly.

In [26]:
#import all three versions of the file ,CDF, csv, and binary *.dat
metadata, baytcsv = sonde3.sonde("tests/ysi_test_files/0108BAYT.csv", pytz.timezone('US/Central'))
metadata, baytdat = sonde3.sonde("tests/ysi_test_files/0108BAYT.dat", pytz.timezone('US/Central'))
metadata, baytCDF = sonde3.sonde("tests/ysi_test_files/0108BAYT.CDF", pytz.timezone('US/Central'))

In [27]:
#cut the first & last row from each file.
pandas.concat([baytCDF.iloc[0:1], baytdat.iloc[0:1], baytcsv.iloc[0:1] , \
               baytCDF.iloc[-1:], baytdat.iloc[-1:], baytcsv.iloc[-1:]], axis=0,join="inner")

,Datetime_(UTC),water_temp_C,water_specific_conductivity_mS/cm,water_depth_m_nonvented,water_DO_%,water_conductivity_mS/cm,water_DO_mgl,water_salinity_PSU
0,2008-01-29 15:00:33+00:00,22.560000,-0.0,-0.0,103.200000,-0.0,8.927547,0.011370
0,2008-01-29 15:00:33+00:00,22.562561,-0.0,-0.0,103.227234,-0.0,8.929467,0.011371
0,2008-01-29 15:00:00+00:00,22.560000,0.0,-0.0,103.200000,0.0,8.927547,0.011370
1061,2008-03-13 19:00:33+00:00,22.510000,-0.0,-0.0,104.300000,-0.0,9.031326,0.011350
1061,2008-03-13 19:00:33+00:00,22.510376,-0.0,-0.0,104.284668,-0.0,9.029934,0.011350
1061,2008-03-13 19:00:00+00:00,22.510000,0.0,-0.0,104.300000,0.0,9.031326,0.011350


In [28]:
metadata, df = sonde3.sonde("tests/lowell_test_files/1708018_3Basin.csv", twdbparams=True)

C:\Users\ETurner\Desktop\python\sonde3\sonde3\sonde.py:34: UserWarning: Info: No time zone was set for file, assuming records are recorded in CST
  metadata, df = formats.read_lowell(filename,  tzinfo,',')
C:\Users\ETurner\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [29]:
df.head()

,Datetime_(UTC),water_speed,water_bearing,northward_water_velocity,eastward_water_velocity,water_temperature
3,2018-02-23 12:05:00+00:00,25.70,27.28,22.84,11.78,10.3240
4,2018-02-23 12:10:00+00:00,25.70,26.36,23.03,11.41,10.3396
5,2018-02-23 12:15:00+00:00,25.70,26.17,23.07,11.33,10.3539
6,2018-02-23 12:20:00+00:00,25.70,25.76,23.15,11.17,10.3767
7,2018-02-23 12:25:00+00:00,25.83,26.96,23.02,11.71,10.3966


In [30]:
metadata, df = sonde3.sonde("T:/baysestuaries/Data/WQData/sites/upbaff/twdb_wq_upbaff_provisional-2.csv")

C:\Users\ETurner\Desktop\python\sonde3\sonde3\sonde.py:28: UserWarning: Info: No time zone was set for file, assuming records are recorded in CST
  metadata, df = formats.read_ysi_ascii(filename,  tzinfo,',',[0])


In [31]:
df.head()

,Datetime_(UTC),water_depth_m_nonvented,water_DO_mgl,water_conductivity_mS/cm,water_pH,water_temp_C,water_specific_conductivity_mS/cm,water_salinity_PSU
0,1998-10-23 18:00:00+00:00,0.0,7.53,28.5,8.02,20.61,31.108409,19.382579
1,1998-10-23 19:00:00+00:00,0.0,7.19,28.5,8.04,20.63,31.095443,19.373523
2,1998-10-23 20:00:00+00:00,0.0,6.85,28.7,8.05,20.86,31.164285,19.418600
3,1998-10-23 21:00:00+00:00,0.0,6.81,28.9,8.06,21.00,31.290602,19.503839
4,1998-10-23 22:00:00+00:00,0.0,7.30,28.7,8.06,21.16,30.971580,19.283727


In [32]:
metadata, df = sonde3.sonde("T:/baysestuaries/Data/WDFT-Coastal/db_archive/deployment/0518-GEA-s_17H104159_050818_080000.csv", twdbparams=True)

C:\Users\ETurner\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [33]:
metadata.head()

,Manufacturer,Instrument_Serial_Number,Sensor_Serial_Numbers,Model,Station,Deployment_Setup_Time,Deployment_Start_Time,Deployment_Stop_Time,Filename,User,Averaging,Firmware,Sensor_Firmware
0,YSI,17H104159,17H101445;17G101401;17H101512;17H100178,EXO,GEA-S,NaN,2018-05-08 11:30:01,2018-06-08 19:30:01,NaN,Evan Turner,Default,3.0.5,3.0.0;3.0.5;3.0.0;3.0.0


In [34]:
type(metadata)

pandas.core.frame.DataFrame

In [35]:
metadata.head()

,Manufacturer,Instrument_Serial_Number,Sensor_Serial_Numbers,Model,Station,Deployment_Setup_Time,Deployment_Start_Time,Deployment_Stop_Time,Filename,User,Averaging,Firmware,Sensor_Firmware
0,YSI,17H104159,17H101445;17G101401;17H101512;17H100178,EXO,GEA-S,NaN,2018-05-08 11:30:01,2018-06-08 19:30:01,NaN,Evan Turner,Default,3.0.5,3.0.0;3.0.5;3.0.0;3.0.0


In [36]:
df.isna().any()

Datetime_(UTC)                               False
sonde_fault_code                             False
instrument_battery_voltage                   False
sonde_cable_power_voltage                    False
water_dissolved_oxygen_percent_saturation    False
water_dissolved_oxygen_concentration         False
water_temperature                            False
water_nLF_conductivity_uS/cm                 False
water_tds_mgl                                False
water_pressure_abs                           False
water_depth_nonvented                        False
water_electrical_conductivity                False
water_specific_conductance                   False
seawater_salinity                            False
dtype: bool

In [37]:
df

,Datetime_(UTC),sonde_fault_code,instrument_battery_voltage,sonde_cable_power_voltage,water_dissolved_oxygen_percent_saturation,water_dissolved_oxygen_concentration,water_temperature,water_nLF_conductivity_uS/cm,water_tds_mgl,water_pressure_abs,water_depth_nonvented,water_electrical_conductivity,water_specific_conductance,seawater_salinity
21,2018-05-08 11:30:01,0.0,3.0,1.6,45.4,3.996738,21.600,369.0,238.0,0.321,0.226,0.3429,0.3667,0.177180
22,2018-05-08 11:40:01,0.0,3.0,1.6,45.5,4.011911,21.518,369.0,238.0,0.321,0.226,0.3423,0.3666,0.177173
23,2018-05-08 11:50:01,0.0,3.0,1.6,45.3,4.000014,21.444,369.1,238.0,0.324,0.228,0.3418,0.3667,0.177188
24,2018-05-08 12:00:01,0.0,3.0,1.6,45.3,4.005845,21.369,369.1,238.0,0.323,0.228,0.3412,0.3666,0.177154
25,2018-05-08 12:10:01,0.0,3.0,1.6,45.3,4.011457,21.297,369.1,238.0,0.324,0.228,0.3407,0.3666,0.177161
26,2018-05-08 12:20:01,0.0,3.0,1.6,45.5,4.034896,21.224,369.2,238.0,0.325,0.229,0.3402,0.3666,0.177171
27,2018-05-08 12:30:01,0.0,3.0,1.6,45.3,4.021155,21.173,369.2,238.0,0.327,0.230,0.3398,0.3666,0.177152
28,2018-05-08 12:40:01,0.0,3.0,1.6,45.6,4.051101,21.131,369.2,238.0,0.330,0.232,0.3395,0.3666,0.177152
29,2018-05-08 12:50:01,0.0,3.0,1.6,47.3,4.205653,21.088,369.4,238.0,0.330,0.233,0.3394,0.3668,0.177260
30,2018-05-08 13:00:01,0.0,3.0,1.6,48.4,4.307661,21.038,369.5,238.0,0.336,0.237,0.3391,0.3668,0.177290


In [38]:
pandas.__version__

'0.23.3'

In [39]:
import pandas as pd

df = pd.DataFrame({ 'event_time': ['gouda', 'stilton', 'gruyere']})

urls = pd.DataFrame({'url': ['g(.*)']})   # With a capturing group, there is a UserWarning
urls = pd.DataFrame({'url': ['g.*']})   # Without a capturing group, there is no UserWarning. Uncommenting this line avoids the UserWarning.

substr = urls.url.values.tolist()
df[df['event_time'].str.contains('|'.join(substr), regex=True)]

,event_time
0,gouda
2,gruyere
